In [1]:
import os

# takes care of annoying TF-GPU warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# remove useless Tensorflow warning:
# WARNING:absl:Found untraced functions such as _update_step_xla, lstm_cell_1_layer_call_fn, 
# lstm_cell_1_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, 
# lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). 
# These functions will not be directly callable after loading.
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
# very useful for managing wandb runs: https://stackoverflow.com/questions/71106179/log-two-model-runs-with-keras-wandb
import wandb
from wandb.keras import WandbCallback
os.environ["WANDB_SILENT"] = "true"

In [3]:
import multi_classifier_utils as mc_u

#### RNN (BiLSTM): Formula Label Prediction (multi-label, all features)

In [4]:
import numpy as np 
import pandas as pd
from pathlib import Path 
import ast

import tensorflow as tf
import keras_tuner as kt

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import datetime

tfds.disable_progress_bar()
wandb_project_name = "multi_label_formula_classification"

In [5]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_"+metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_"+metric])

#### Import Data and Preprocess Dataframe

In [6]:
def preprocess_data(corpus,
                    irrelevant_features=["mtype",]):
    # drop irrelevant columns
    corpus.drop(irrelevant_features, inplace=True, axis=1)

    def cell_str_to_list(cell_val):
        return ast.literal_eval(cell_val)

    # filter strings
    def process_cell(cell_str):
        stripped_f_str = cell_str[1:-1].replace("\\\\", "\\")
        f_list = stripped_f_str.split(",")
        f_list = [token.replace("'", "").replace(" ", "") for token in f_list]
        f_list = ["{" if token == "\\{" else token for token in f_list]
        f_list = ["}" if token == "\\}" else token for token in f_list]
        cell_str = " ".join(f_list)
        return cell_str

    corpus["type_tokens"] = corpus["type_tokens"].map(process_cell)
    corpus["tokens"] = corpus["tokens"].map(process_cell)
    corpus["mtype_one_hot"] = corpus["mtype_one_hot"].map(cell_str_to_list)
    corpus["labels"] = corpus["labels"].map(cell_str_to_list)
    corpus = corpus.loc[(corpus["tokens"].str.len() > 0) & (corpus["tokens"] != " ")]


In [7]:
#print(os.getcwd())
data_p = Path("../data/") / "multi_class_unbalanced_data_TOKENIZED_V1.csv"
data = pd.read_csv(data_p)
preprocess_data(data)
data.head()

,tokens,type_tokens,labels,labels_str,mtype_one_hot
0,f,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
1,j : \mathbb{N} \rightarrow [ 0 1 ],func_def func_name __ANON_1 COLON mapping set_...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
2,b,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
3,mathbb{Q,set_constant SET_BASIC,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[1, 0, 0]"
4,f,func_name __ANON_1,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['functions', 'elementary-set-theory']","[0, 0, 1]"


In [8]:
print(data["type_tokens"].map(lambda x: len((x.split(" ")))).max())
print(data["tokens"].map(lambda x: len((x.split(" ")))).max())

254
252


In [9]:
# ordinary datasets
SMALL_TRAIN_SIZE = 24620 - 2460
SMALL_TEST_SIZE = 2460
LARGE_TRAIN_SIZE = 106523 - 10650
LARGE_TEST_SIZE = 10650
# compact datasets
NUM_CLASSES = 40

labels_array = np.array(data["labels"].to_list())
m_type_array = np.array(data["mtype_one_hot"].to_list())

dataset4_all_features = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"],m_type_array), name="data")
dataset3_tokens_types = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"]), name="data")
dataset2_types = tf.data.Dataset.from_tensor_slices((data["type_tokens"]), name="data")
dataset1_tokens = tf.data.Dataset.from_tensor_slices((data["tokens"]), name="data")

labels_ds = tf.data.Dataset.from_tensor_slices(labels_array, name="label")
#data_as_ds = tf.data.Dataset.zip((dat_as_ds, labels_ds))

dataset1_tokens_l = tf.data.Dataset.zip((dataset1_tokens, labels_ds))
dataset2_types_l = tf.data.Dataset.zip((dataset2_types, labels_ds))
dataset3_tokens_types_l = tf.data.Dataset.zip((dataset3_tokens_types, labels_ds))
dataset4_all_features_l = tf.data.Dataset.zip((dataset4_all_features, labels_ds))

In [10]:
test_dataset1 = dataset1_tokens_l.take(SMALL_TEST_SIZE)
train_dataset1 = dataset1_tokens_l.skip(SMALL_TEST_SIZE)
test_dataset2 = dataset2_types_l.take(SMALL_TEST_SIZE)
train_dataset2 = dataset2_types_l.skip(SMALL_TEST_SIZE)
test_dataset3 = dataset3_tokens_types_l.take(SMALL_TEST_SIZE)
train_dataset3 = dataset3_tokens_types_l.skip(SMALL_TEST_SIZE)
test_dataset4 = dataset4_all_features_l.take(SMALL_TEST_SIZE)
train_dataset4 = dataset4_all_features_l.skip(SMALL_TEST_SIZE)

##### Setup and Data Preparation

In [11]:
for (example_token, example_type, example_m_type), label in train_dataset4.take(5):
    print("text: ", example_token.numpy())
    print("type: ", example_type.numpy())
    print("m_type: ", example_m_type.numpy())
    print("label: ", label.numpy())

text:  b'{ 2   4 }'
type:  b'explset L_BRACE_LITERAL set_enumeration item __ANON_3 COMMA item __ANON_3 R_BRACE_LITERAL'
m_type:  [1 0 0]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : I \\to X'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f ( a ) = 1'
type:  b'func_expr func_name_arg func_name __ANON_1 L_PAREN __ANON_1 R_PAREN EQUAL expr_atom __ANON_3'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'x'
type:  b'func_name __ANON_1'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : X \\to Y'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [12]:
BUFFER_SIZE = 2000
BATCH_SIZE = 64
STEPS_PER_EPOCH = np.floor(SMALL_TRAIN_SIZE/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.floor(SMALL_TEST_SIZE/BATCH_SIZE)
#train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
#test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [13]:
test_dataset1 = test_dataset1.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset1 = train_dataset1.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset2 = test_dataset2.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset2 = train_dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset3 = test_dataset3.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset3 = train_dataset3.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset4 = test_dataset4.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset4 = train_dataset4.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

##### Text Encoding

In [14]:
# for int encoder
TYPE_TOKENS_MAX_SEQ_LEN = 260
TOKENS_MAX_SEQ_LEN = 260


# for other encoders 
TYPE_TOKENS_PAD_TO_MAX_TOKENS = 80
TOKENS_PAD_TO_MAX_TOKENS = 200
BIGRAM_PAD_TO_MAX_TOKENS = 350

In [15]:
def adapt_encoder(encoder, mode, dataset, dataset_type):
    if mode == "token": 

        if dataset_type == 1:
            encoder.adapt(dataset.map(lambda inputs, label: inputs))
        elif dataset_type == 2:
            ...
        elif dataset_type == 3:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text
        elif dataset_type == 4:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text  
    elif mode == "type":
        
        if dataset_type == 1:
            ...
        elif dataset_type == 2:
            encoder.adapt(dataset.map(lambda inputs, label: inputs))
        elif dataset_type == 3:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text
        elif dataset_type == 4:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text

    return encoder

In [16]:
def create_encoder(output_mode_str, n_grams, mode, dataset, dataset_type):
    if output_mode_str == "int":
        VOCAB_SIZE = 200
        if mode == "token":
            max_seq_len = TOKENS_MAX_SEQ_LEN
        elif mode == "type":
            max_seq_len = TYPE_TOKENS_MAX_SEQ_LEN

        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            output_sequence_length = max_seq_len,
            split="whitespace",
            max_tokens=VOCAB_SIZE)
        #TODO: adapt for different inputs
        encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
        return encoder
    
    if output_mode_str == "count" and n_grams == 2:
        max_seq_len = BIGRAM_PAD_TO_MAX_TOKENS
        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            pad_to_max_tokens = max_seq_len,
            split="whitespace",
            max_tokens=max_seq_len)
        
        
        encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
        return encoder
    
    if mode == "token":
        max_seq_len = TOKENS_PAD_TO_MAX_TOKENS
    elif mode == "type":
        max_seq_len = TYPE_TOKENS_PAD_TO_MAX_TOKENS

    encoder = tf.keras.layers.TextVectorization(
        standardize=None,
        output_mode=output_mode_str,
        ngrams = n_grams,
        pad_to_max_tokens = max_seq_len,
        split="whitespace",
        max_tokens=max_seq_len)
    #TODO: adapt for different inputs
    encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
    return encoder


    

##### Representation 1: Use integer indices encoding

In [17]:
encoder_int_tokens = create_encoder("int", None, "token", train_dataset1, 1)
encoder_int_types = create_encoder("int", None, "type", train_dataset2, 2)

vocab_tokens = np.array(encoder_int_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_int_tokens.get_vocabulary())
vocab_types = np.array(encoder_int_types.get_vocabulary())
vocab_size_types = len(encoder_int_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
tokens (voc size):  128
types (voc size):  61


In [18]:
encoded_example_token = encoder_int_tokens(example_token).numpy()
encoded_example_types = encoder_int_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[ 2 11 26 14 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
(260,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[15  5  3 13 14  6 16  6  0  0  

##### Representation 2: Count Vectorizer

In [19]:
encoder_count_tokens = create_encoder("count", None, "token", train_dataset1, 1)
encoder_count_types = create_encoder("count", None, "type", train_dataset2, 2)

vocab_tokens = np.array(encoder_count_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_count_tokens.get_vocabulary())
vocab_types = np.array(encoder_count_types.get_vocabulary())
vocab_size_types = len(encoder_count_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

tokens (voc size):  127
types (voc size):  60


In [20]:
encoded_example_token = encoder_count_tokens(example_token).numpy()
encoded_example_types = encoder_count_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
(200,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[0. 0. 1. 0. 1. 2. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

##### Model (RNN(BiLSTM))

 **Train the model**

#### Experiment 1: Use integer indices for encoding tokens

##### Model 1: ONLY TOKENS
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens1 = create_encoder("int", None, "token", train_dataset1, 1)
model_builder1 = mc_u.create_model_builder1_RNN(NUM_CLASSES, encoder_int_tokens1, tokens_input_len)

In [ ]:
tuner = kt.Hyperband(model_builder1,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=15,
                     factor=3,
                     directory='meta_dir/model1_rnn',
                     project_name='model1_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset1,
             epochs=30,
             validation_data=test_dataset1,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 50
model1 = mc_u.create_model1_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_int_tokens1, tokens_input_len)
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
     
    "configuration": "multi_only_tokens",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1.fit(train_dataset1, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset1,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

##### Model 2: ONLY TYPES
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_types2 = create_encoder("int", None, "type", train_dataset2, 2)    
model_builder2 = mc_u.create_model_builder2_RNN(NUM_CLASSES, encoder_int_types2,type_input_len)

In [ ]:
tuner = kt.Hyperband(model_builder2,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=15,
                     factor=3,
                     directory='meta_dir/model2_rnn',
                     project_name='model2_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


INFO:tensorflow:Reloading Tuner from meta_dir/model2_rnn/model2_rnn/tuner0.json


In [ ]:
tuner.search(train_dataset2,
             epochs=30,
             validation_data=test_dataset2,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.01
64
64
32


Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 50
optimal_lr = 0.0001
model2 = mc_u.create_model2_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_int_types2, type_input_len)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_only_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2.fit(train_dataset2, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset2,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
346/346 [==============================] - ETA: 0s - loss: 0.5069 - accuracy: 0.0928 - recall: 0.0174INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


346/346 [==============================] - 66s 177ms/step - loss: 0.5069 - accuracy: 0.0928 - recall: 0.0174 - val_loss: 0.3014 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
346/346 [==============================] - ETA: 0s - loss: 0.2018 - accuracy: 0.4463 - recall: 0.0496INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


346/346 [==============================] - 59s 172ms/step - loss: 0.2018 - accuracy: 0.4463 - recall: 0.0496 - val_loss: 0.2003 - val_accuracy: 0.0000e+00 - val_recall: 0.0034
Epoch 3/50
346/346 [==============================] - ETA: 0s - loss: 0.1611 - accuracy: 0.4315 - recall: 0.0450INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


...

346/346 [==============================] - 60s 174ms/step - loss: 0.1298 - accuracy: 0.5937 - recall: 0.1797 - val_loss: 0.1344 - val_accuracy: 0.5070 - val_recall: 0.0478
Epoch 49/50
346/346 [==============================] - ETA: 0s - loss: 0.1297 - accuracy: 0.5959 - recall: 0.1794INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


346/346 [==============================] - 61s 177ms/step - loss: 0.1297 - accuracy: 0.5959 - recall: 0.1794 - val_loss: 0.1343 - val_accuracy: 0.5074 - val_recall: 0.0485
Epoch 50/50
346/346 [==============================] - ETA: 0s - loss: 0.1296 - accuracy: 0.5942 - recall: 0.1792INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_202734-z9tb9ipf/files/model-best/assets


346/346 [==============================] - 67s 192ms/step - loss: 0.1296 - accuracy: 0.5942 - recall: 0.1792 - val_loss: 0.1342 - val_accuracy: 0.5074 - val_recall: 0.0485


##### Model 3: TOKENS AND TYPES
Find best hyperparameters

In [21]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens3 = create_encoder("int", None, "token", train_dataset3, 3)
encoder_int_types3 = create_encoder("int", None, "type", train_dataset3, 3)
model_builder3 = mc_u.create_model_builder3_RNN(NUM_CLASSES, encoder_int_tokens3,encoder_int_types3, tokens_input_len, type_input_len)

In [22]:
tuner = kt.Hyperband(model_builder3,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='meta_dir/model3_rnn',
                     project_name='model3_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

INFO:tensorflow:Reloading Tuner from meta_dir/model3_rnn/model3_rnn/tuner0.json


In [23]:
tuner.search(train_dataset3,
             epochs=10,
             validation_data=test_dataset3,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

INFO:tensorflow:Oracle triggered exit


In [24]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.001
32
32
32


Train with best hyperparameters

In [25]:
NUM_EPOCHS = 50
model3 = mc_u.create_model3_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_int_tokens3, encoder_int_types3, tokens_input_len, type_input_len)
model3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [26]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_tokens_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model3.fit(train_dataset3, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset3,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
346/346 [==============================] - ETA: 0s - loss: 0.2025 - accuracy: 0.6179 - recall: 0.2845INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


346/346 [==============================] - 81s 213ms/step - loss: 0.2025 - accuracy: 0.6179 - recall: 0.2845 - val_loss: 0.2389 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
346/346 [==============================] - ETA: 0s - loss: 0.1305 - accuracy: 0.6706 - recall: 0.3047INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


346/346 [==============================] - 72s 207ms/step - loss: 0.1305 - accuracy: 0.6706 - recall: 0.3047 - val_loss: 0.2162 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
346/346 [==============================] - 53s 154ms/step - loss: 0.1304 - accuracy: 0.6634 - recall: 0.2867 - val_loss: 0.2172 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
346/346 [==============================] - ETA: 0s - loss: 0.1284 - accuracy: 0.6807 - recall: 0.3068INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


...

346/346 [==============================] - 77s 222ms/step - loss: 0.1217 - accuracy: 0.6765 - recall: 0.2793 - val_loss: 0.1478 - val_accuracy: 0.5724 - val_recall: 0.0117
Epoch 42/50
346/346 [==============================] - 59s 169ms/step - loss: 0.1184 - accuracy: 0.7144 - recall: 0.3030 - val_loss: 0.1603 - val_accuracy: 0.3898 - val_recall: 0.0320
Epoch 43/50
346/346 [==============================] - ETA: 0s - loss: 0.1212 - accuracy: 0.6797 - recall: 0.2929INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


346/346 [==============================] - 74s 213ms/step - loss: 0.1212 - accuracy: 0.6797 - recall: 0.2929 - val_loss: 0.1466 - val_accuracy: 0.5822 - val_recall: 0.0255
Epoch 44/50
346/346 [==============================] - 57s 165ms/step - loss: 0.1217 - accuracy: 0.6752 - recall: 0.2674 - val_loss: 0.1584 - val_accuracy: 0.3902 - val_recall: 0.0219
Epoch 45/50
346/346 [==============================] - 57s 166ms/step - loss: 0.1222 - accuracy: 0.6572 - recall: 0.2758 - val_loss: 0.1504 - val_accuracy: 0.3466 - val_recall: 0.0291
Epoch 46/50
346/346 [==============================] - 57s 164ms/step - loss: 0.1217 - accuracy: 0.6609 - recall: 0.2606 - val_loss: 0.1587 - val_accuracy: 0.3668 - val_recall: 0.0424
Epoch 47/50
346/346 [==============================] - 57s 164ms/step - loss: 0.1222 - accuracy: 0.6615 - recall: 0.2688 - val_loss: 0.1517 - val_accuracy: 0.3590 - val_recall: 0.0428
Epoch 48/50
346/346 [==============================] - 57s 165ms/step - loss: 0.1209 - accur

INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_083141-cc97whmn/files/model-best/assets


346/346 [==============================] - 73s 211ms/step - loss: 0.1223 - accuracy: 0.6562 - recall: 0.2629 - val_loss: 0.1410 - val_accuracy: 0.6460 - val_recall: 0.0451
Epoch 50/50
346/346 [==============================] - 57s 165ms/step - loss: 0.1214 - accuracy: 0.6755 - recall: 0.2742 - val_loss: 0.1484 - val_accuracy: 0.3639 - val_recall: 0.0210


##### Model 4: TOKENS AND TYPES AND SEM MATH LABELS
Find best hyperparameters

In [27]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens4 = create_encoder("int", None, "token", train_dataset4, 4)
encoder_int_types4 = create_encoder("int", None, "type", train_dataset4, 4)
model_builder4 = mc_u.create_model_builder4_RNN(NUM_CLASSES, encoder_int_tokens4,encoder_int_types4, tokens_input_len, type_input_len, "int")

In [28]:
tuner = kt.Hyperband(model_builder4,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='meta_dir/model4_rnn',
                     project_name='model4_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [29]:
tuner.search(train_dataset4,
             epochs=10,
             validation_data=test_dataset4,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 27m 33s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.02302631549537182
Total elapsed time: 04h 21m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [30]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.001
32
32
96


Train with best hyperparameters

In [31]:
NUM_EPOCHS = 50
model4 = mc_u.create_model4_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_int_tokens4, encoder_int_types4, tokens_input_len, type_input_len, "int")
model4.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [32]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_all_features",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model4.fit(train_dataset4, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset4,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
346/346 [==============================] - ETA: 0s - loss: 0.1748 - accuracy: 0.7570 - recall_1: 0.3469INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


346/346 [==============================] - 69s 189ms/step - loss: 0.1748 - accuracy: 0.7570 - recall_1: 0.3469 - val_loss: 0.2372 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/50
346/346 [==============================] - ETA: 0s - loss: 0.1232 - accuracy: 0.7226 - recall_1: 0.3523INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


346/346 [==============================] - 65s 188ms/step - loss: 0.1232 - accuracy: 0.7226 - recall_1: 0.3523 - val_loss: 0.2340 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/50
346/346 [==============================] - ETA: 0s - loss: 0.1255 - accuracy: 0.6958 - recall_1: 0.3257INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


...

346/346 [==============================] - 64s 186ms/step - loss: 0.1191 - accuracy: 0.6832 - recall_1: 0.2979 - val_loss: 0.1553 - val_accuracy: 0.1546 - val_recall_1: 0.0284
Epoch 39/50
346/346 [==============================] - ETA: 0s - loss: 0.1172 - accuracy: 0.7026 - recall_1: 0.3119INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


346/346 [==============================] - 65s 187ms/step - loss: 0.1172 - accuracy: 0.7026 - recall_1: 0.3119 - val_loss: 0.1513 - val_accuracy: 0.3207 - val_recall_1: 0.0144
Epoch 40/50
346/346 [==============================] - 52s 152ms/step - loss: 0.1181 - accuracy: 0.6917 - recall_1: 0.2950 - val_loss: 0.1628 - val_accuracy: 0.1271 - val_recall_1: 0.0194
Epoch 41/50
346/346 [==============================] - ETA: 0s - loss: 0.1190 - accuracy: 0.6772 - recall_1: 0.2917INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_134422-2jdpsz1w/files/model-best/assets


346/346 [==============================] - 65s 189ms/step - loss: 0.1190 - accuracy: 0.6772 - recall_1: 0.2917 - val_loss: 0.1481 - val_accuracy: 0.3569 - val_recall_1: 0.0183
Epoch 42/50
346/346 [==============================] - 49s 143ms/step - loss: 0.1184 - accuracy: 0.6853 - recall_1: 0.2833 - val_loss: 0.1780 - val_accuracy: 0.2730 - val_recall_1: 0.0194
Epoch 43/50
346/346 [==============================] - 49s 142ms/step - loss: 0.1194 - accuracy: 0.6777 - recall_1: 0.2872 - val_loss: 0.1632 - val_accuracy: 0.2726 - val_recall_1: 0.0248
Epoch 44/50
346/346 [==============================] - 48s 138ms/step - loss: 0.1202 - accuracy: 0.6614 - recall_1: 0.2788 - val_loss: 0.1979 - val_accuracy: 0.0794 - val_recall_1: 0.0248
Epoch 45/50
346/346 [==============================] - 48s 138ms/step - loss: 0.1198 - accuracy: 0.6722 - recall_1: 0.2787 - val_loss: 0.1549 - val_accuracy: 0.2603 - val_recall_1: 0.0325
Epoch 46/50
346/346 [==============================] - 48s 138ms/step - 

#### Experiment 2: Use count vectorizer

##### Model 1: ONLY TOKENS
Find best hyperparameters

In [21]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens1 = create_encoder("count", None, "token", train_dataset1, 1)
model_builder1c = mc_u.create_model_builder1_RNN(NUM_CLASSES, encoder_count_tokens1,tokens_input_len)

In [22]:
tuner = kt.Hyperband(model_builder1c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='meta_dir/model1c_rnn',
                     project_name='model1c_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [23]:
tuner.search(train_dataset1,
             epochs=10,
             validation_data=test_dataset1,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 05m 46s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.4597039520740509
Total elapsed time: 01h 27m 55s
INFO:tensorflow:Oracle triggered exit


In [24]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.0001
64
96
96


Train with best hyperparameters

In [25]:
NUM_EPOCHS = 50
optimal_lr = 0.0001
model1c = mc_u.create_model1_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_count_tokens, tokens_input_len)
model1c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [26]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_only_tokens",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1c.fit(train_dataset1, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset1,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
346/346 [==============================] - ETA: 0s - loss: 0.4780 - accuracy: 0.4781 - recall_1: 0.0577INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


346/346 [==============================] - 79s 212ms/step - loss: 0.4780 - accuracy: 0.4781 - recall_1: 0.0577 - val_loss: 0.3020 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/50
346/346 [==============================] - ETA: 0s - loss: 0.2471 - accuracy: 0.5140 - recall_1: 0.1170INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


346/346 [==============================] - 73s 210ms/step - loss: 0.2471 - accuracy: 0.5140 - recall_1: 0.1170 - val_loss: 0.2092 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0020
Epoch 3/50
346/346 [==============================] - ETA: 0s - loss: 0.2150 - accuracy: 0.4856 - recall_1: 0.1007INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


...

346/346 [==============================] - 72s 209ms/step - loss: 0.1385 - accuracy: 0.5152 - recall_1: 0.0599 - val_loss: 0.1404 - val_accuracy: 0.6225 - val_recall_1: 2.2548e-04
Epoch 39/50
346/346 [==============================] - ETA: 0s - loss: 0.1384 - accuracy: 0.5159 - recall_1: 0.0608INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


346/346 [==============================] - 73s 212ms/step - loss: 0.1384 - accuracy: 0.5159 - recall_1: 0.0608 - val_loss: 0.1402 - val_accuracy: 0.6225 - val_recall_1: 2.2548e-04
Epoch 40/50
346/346 [==============================] - 57s 164ms/step - loss: 0.1382 - accuracy: 0.5150 - recall_1: 0.0613 - val_loss: 0.1404 - val_accuracy: 0.6225 - val_recall_1: 2.2548e-04
Epoch 41/50
346/346 [==============================] - 57s 163ms/step - loss: 0.1383 - accuracy: 0.5144 - recall_1: 0.0613 - val_loss: 0.1404 - val_accuracy: 0.6213 - val_recall_1: 2.2548e-04
Epoch 42/50
346/346 [==============================] - ETA: 0s - loss: 0.1383 - accuracy: 0.5154 - recall_1: 0.0612INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_155959-a4pm31rv/files/model-best/assets


346/346 [==============================] - 80s 232ms/step - loss: 0.1383 - accuracy: 0.5154 - recall_1: 0.0612 - val_loss: 0.1393 - val_accuracy: 0.6538 - val_recall_1: 2.2548e-04
Epoch 43/50
346/346 [==============================] - 63s 182ms/step - loss: 0.1379 - accuracy: 0.5204 - recall_1: 0.0638 - val_loss: 0.1422 - val_accuracy: 0.6287 - val_recall_1: 6.7644e-04
Epoch 44/50
346/346 [==============================] - 56s 163ms/step - loss: 0.1381 - accuracy: 0.5147 - recall_1: 0.0595 - val_loss: 0.1408 - val_accuracy: 0.6287 - val_recall_1: 2.2548e-04
Epoch 45/50
346/346 [==============================] - 57s 163ms/step - loss: 0.1379 - accuracy: 0.5187 - recall_1: 0.0616 - val_loss: 0.1393 - val_accuracy: 0.6242 - val_recall_1: 2.2548e-04
Epoch 46/50
346/346 [==============================] - 57s 164ms/step - loss: 0.1379 - accuracy: 0.5152 - recall_1: 0.0625 - val_loss: 0.1400 - val_accuracy: 0.6213 - val_recall_1: 2.2548e-04
Epoch 47/50
346/346 [==============================]

##### Model 2: ONLY TYPES
Find best hyperparameters

In [41]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_types = create_encoder("count", None, "type", train_dataset2, 2)
model_builder2c = mc_u.create_model_builder2_RNN(NUM_CLASSES, encoder_count_types, type_input_len)

In [42]:
tuner = kt.Hyperband(model_builder2c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='meta_dir/model2c_rnn',
                     project_name='model2c_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

INFO:tensorflow:Reloading Tuner from meta_dir/model2c_rnn/model2c_rnn/tuner0.json


In [43]:
tuner.search(train_dataset2,
             epochs=10,
             validation_data=test_dataset2,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 07m 44s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 01h 09m 28s
INFO:tensorflow:Oracle triggered exit


In [44]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.0001
32
32
128


Train with best hyperparameters

In [45]:
NUM_EPOCHS = 50
optimal_lr = 0.0001
model2c = mc_u.create_model2_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_count_types, type_input_len)
model2c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [46]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_only_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2c.fit(train_dataset2, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset2,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
345/346 [============================>.] - ETA: 0s - loss: 0.4961 - accuracy: 0.3727 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


346/346 [==============================] - 33s 80ms/step - loss: 0.4954 - accuracy: 0.3725 - recall_1: 0.0000e+00 - val_loss: 0.2488 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/50
345/346 [============================>.] - ETA: 0s - loss: 0.1845 - accuracy: 0.4199 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


346/346 [==============================] - 26s 75ms/step - loss: 0.1845 - accuracy: 0.4196 - recall_1: 0.0000e+00 - val_loss: 0.1809 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/50
345/346 [============================>.] - ETA: 0s - loss: 0.1549 - accuracy: 0.3818 - recall_1: 0.0128INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


...

346/346 [==============================] - 27s 78ms/step - loss: 0.1477 - accuracy: 0.3290 - recall_1: 0.0050 - val_loss: 0.1576 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 32/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1473 - accuracy: 0.3391 - recall_1: 0.0049 - val_loss: 0.1599 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 33/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1475 - accuracy: 0.3412 - recall_1: 0.0071 - val_loss: 0.1577 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 34/50
345/346 [============================>.] - ETA: 0s - loss: 0.1472 - accuracy: 0.3509 - recall_1: 0.0077INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


346/346 [==============================] - 27s 79ms/step - loss: 0.1472 - accuracy: 0.3507 - recall_1: 0.0077 - val_loss: 0.1576 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 35/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1469 - accuracy: 0.3506 - recall_1: 0.0079 - val_loss: 0.1592 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 36/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1468 - accuracy: 0.3577 - recall_1: 0.0094 - val_loss: 0.1604 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 37/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1467 - accuracy: 0.3609 - recall_1: 0.0122 - val_loss: 0.1605 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 38/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1466 - accuracy: 0.3581 - recall_1: 0.0117 - val_loss: 0.1601 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 39/50
346/346 [================

INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_182403-zai9v8e8/files/model-best/assets


346/346 [==============================] - 28s 79ms/step - loss: 0.1453 - accuracy: 0.3857 - recall_1: 0.0215 - val_loss: 0.1570 - val_accuracy: 0.0045 - val_recall_1: 0.0000e+00
Epoch 48/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1450 - accuracy: 0.3917 - recall_1: 0.0238 - val_loss: 0.1573 - val_accuracy: 0.0045 - val_recall_1: 0.0000e+00
Epoch 49/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1446 - accuracy: 0.3956 - recall_1: 0.0271 - val_loss: 0.1572 - val_accuracy: 0.0045 - val_recall_1: 0.0000e+00
Epoch 50/50
346/346 [==============================] - 11s 33ms/step - loss: 0.1446 - accuracy: 0.4033 - recall_1: 0.0255 - val_loss: 0.1572 - val_accuracy: 0.0045 - val_recall_1: 0.0000e+00


##### Model 3: TOKENS AND TYPES
Find best hyperparameters

In [21]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens3 = create_encoder("count", None, "token", train_dataset3, 3)
encoder_count_types3 = create_encoder("count", None, "type", train_dataset3, 3)
model_builder3c = mc_u.create_model_builder3_RNN(NUM_CLASSES, encoder_count_tokens3,encoder_count_types3, tokens_input_len, type_input_len)

In [22]:
tuner = kt.Hyperband(model_builder3c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='meta_dir/model3c_rnn',
                     project_name='model3c_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

INFO:tensorflow:Reloading Tuner from meta_dir/model3c_rnn/model3c_rnn/tuner0.json


In [23]:
tuner.search(train_dataset3,
             epochs=10,
             validation_data=test_dataset3,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 19m 46s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 34m 43s
INFO:tensorflow:Oracle triggered exit


In [24]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.01
96
96
64


Train with best hyperparameters

In [25]:
NUM_EPOCHS = 50
optimal_lr = 0.0001
model3c = mc_u.create_model3_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_count_tokens3, encoder_count_types3, tokens_input_len, type_input_len)
model3c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [26]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_tokens_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model3c.fit(train_dataset3, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset3,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
346/346 [==============================] - ETA: 0s - loss: 0.3918 - accuracy: 0.5928 - recall_1: 0.0636INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


346/346 [==============================] - 100s 275ms/step - loss: 0.3918 - accuracy: 0.5928 - recall_1: 0.0636 - val_loss: 0.2344 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/50
346/346 [==============================] - ETA: 0s - loss: 0.1580 - accuracy: 0.5464 - recall_1: 0.1571INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


346/346 [==============================] - 94s 272ms/step - loss: 0.1580 - accuracy: 0.5464 - recall_1: 0.1571 - val_loss: 0.1991 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/50
346/346 [==============================] - ETA: 0s - loss: 0.1462 - accuracy: 0.5474 - recall_1: 0.1626INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


...

346/346 [==============================] - 100s 290ms/step - loss: 0.1399 - accuracy: 0.5048 - recall_1: 0.0907 - val_loss: 0.1586 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0023
Epoch 46/50
346/346 [==============================] - ETA: 0s - loss: 0.1395 - accuracy: 0.5002 - recall_1: 0.0891INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


346/346 [==============================] - 99s 286ms/step - loss: 0.1395 - accuracy: 0.5002 - recall_1: 0.0891 - val_loss: 0.1570 - val_accuracy: 0.0016 - val_recall_1: 0.0025
Epoch 47/50
346/346 [==============================] - 85s 244ms/step - loss: 0.1390 - accuracy: 0.5133 - recall_1: 0.1033 - val_loss: 0.1591 - val_accuracy: 0.0012 - val_recall_1: 0.0027
Epoch 48/50
346/346 [==============================] - 85s 245ms/step - loss: 0.1387 - accuracy: 0.5162 - recall_1: 0.0999 - val_loss: 0.1576 - val_accuracy: 0.3898 - val_recall_1: 0.0027
Epoch 49/50
346/346 [==============================] - 85s 246ms/step - loss: 0.1383 - accuracy: 0.5175 - recall_1: 0.1004 - val_loss: 0.1597 - val_accuracy: 0.3886 - val_recall_1: 0.0027
Epoch 50/50
346/346 [==============================] - ETA: 0s - loss: 0.1380 - accuracy: 0.5336 - recall_1: 0.1096INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files

INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230317_215056-wu7j07wp/files/model-best/assets


346/346 [==============================] - 100s 290ms/step - loss: 0.1380 - accuracy: 0.5336 - recall_1: 0.1096 - val_loss: 0.1567 - val_accuracy: 0.5555 - val_recall_1: 0.0027


##### Model 4: TOKENS, TYPES, SEM MATH LABELS
Find best hyperparameters

In [21]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens4 = create_encoder("count", None, "token", train_dataset4, 4)
encoder_count_types4 = create_encoder("count", None, "type", train_dataset4, 4)
model_builder4c = mc_u.create_model_builder4_RNN(NUM_CLASSES, encoder_count_tokens4, encoder_count_types4, tokens_input_len, type_input_len, "float")

In [22]:
tuner = kt.Hyperband(model_builder4c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=10,
                     factor=3,
                     directory='meta_dir/model4c_rnn',
                     project_name='model4c_rnn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

INFO:tensorflow:Reloading Tuner from meta_dir/model4c_rnn/model4c_rnn/tuner0.json


In [23]:
tuner.search(train_dataset4,
             epochs=10,
             validation_data=test_dataset4,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 05m 17s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.07113486528396606
Total elapsed time: 01h 37m 01s
INFO:tensorflow:Oracle triggered exit


In [24]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
optimal_lstm_units = best_hps.get("lstm_units")
optimal_dense_units = best_hps.get("dense_units")

print(optimal_lr)
print(optimal_emb_dims)
print(optimal_lstm_units)
print(optimal_dense_units)

0.0001
32
96
32


Train with best hyperparameters

In [25]:
NUM_EPOCHS = 50
optimal_lr = 0.0001
model4c = mc_u.create_model4_RNN(NUM_CLASSES, optimal_emb_dims, optimal_lstm_units, optimal_dense_units, encoder_count_tokens4, encoder_count_types4, tokens_input_len, type_input_len, "float")
model4c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [26]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "lstm_units": optimal_lstm_units, 
    "dense_units": optimal_dense_units,
    "algorithm": "BiLstm",
    "configuration": "multi_all_features",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model4c.fit(train_dataset4, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset4,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/50
346/346 [==============================] - ETA: 0s - loss: 0.4539 - accuracy: 0.0714 - recall_1: 0.0190INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


346/346 [==============================] - 83s 226ms/step - loss: 0.4539 - accuracy: 0.0714 - recall_1: 0.0190 - val_loss: 0.2462 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/50
346/346 [==============================] - ETA: 0s - loss: 0.1764 - accuracy: 0.4296 - recall_1: 0.0336INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


346/346 [==============================] - 77s 223ms/step - loss: 0.1764 - accuracy: 0.4296 - recall_1: 0.0336 - val_loss: 0.1874 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/50
346/346 [==============================] - ETA: 0s - loss: 0.1522 - accuracy: 0.4953 - recall_1: 0.0563INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


...

346/346 [==============================] - 77s 223ms/step - loss: 0.1467 - accuracy: 0.4437 - recall_1: 0.0093 - val_loss: 0.1700 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 19/50
346/346 [==============================] - 62s 179ms/step - loss: 0.1467 - accuracy: 0.4393 - recall_1: 0.0071 - val_loss: 0.1700 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 20/50
346/346 [==============================] - ETA: 0s - loss: 0.1466 - accuracy: 0.4394 - recall_1: 0.0065INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


346/346 [==============================] - 78s 224ms/step - loss: 0.1466 - accuracy: 0.4394 - recall_1: 0.0065 - val_loss: 0.1698 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 21/50
346/346 [==============================] - 62s 180ms/step - loss: 0.1466 - accuracy: 0.4384 - recall_1: 0.0011 - val_loss: 0.1698 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 22/50
346/346 [==============================] - ETA: 0s - loss: 0.1448 - accuracy: 0.4918 - recall_1: 0.0475INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230318_014108-vnzbob5m/files/model-best/assets


346/346 [==============================] - 78s 225ms/step - loss: 0.1448 - accuracy: 0.4918 - recall_1: 0.0475 - val_loss: 0.1660 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 23/50
346/346 [==============================] - 62s 180ms/step - loss: 0.1455 - accuracy: 0.4671 - recall_1: 0.0742 - val_loss: 0.1694 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 24/50
346/346 [==============================] - 62s 179ms/step - loss: 0.1463 - accuracy: 0.4538 - recall_1: 0.0340 - val_loss: 0.1707 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 25/50
346/346 [==============================] - 62s 179ms/step - loss: 0.1462 - accuracy: 0.4496 - recall_1: 0.0174 - val_loss: 0.1717 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 26/50
346/346 [==============================] - 63s 181ms/step - loss: 0.1463 - accuracy: 0.4380 - recall_1: 0.0082 - val_loss: 0.1714 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 27/50
346/346 [===========